In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [10, 8]

import warnings
warnings.filterwarnings('ignore')

# load data

In [2]:
df1 = pd.read_json('/fastcampus-data/orders/orders.json') # origin data

# copy해서 쓰기
df = df1.copy()

In [3]:
# 전체 보고싶음 , 빼놓은 컬럼( 'status_id','delivered','payment_method_id','date_created', 'date_placed', 'date_updated', 'date_closed',)
df = df[['customer_id','date_paid', 'items',
       'delivered', 'cancelled', 'paid', 'hold', 'draft',
          'note']]


# preprocessing: dict 풀기

In [13]:
# 전처리. dict 푸는 코드
# x['$oid'] -> x의 키 값을 가져오라는 뜻!!
df['customer_id'] = df['customer_id'].apply(lambda x: x['$oid'] if type(x)==dict else None)

# date_* dict 풀기
df['date_paid'] = df['date_paid'].apply(lambda x: x['$date'] if type(x)==dict else None)



In [14]:
# items의 빈 리스트 none으로 바꿔주기
df['items'] = df['items'].apply(lambda x: None if x == [] else x)

# label encoding

In [16]:
# boolean type labelencoding

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# 그냥 df encoding
# df.loc[:,'closed'] = le.fit_transform(df_binary['closed'])
df.loc[:,'cancelled'] = le.fit_transform(df['cancelled'])
df.loc[:, 'delivered'] = le.fit_transform(df['delivered'])
df.loc[:,'paid'] = le.fit_transform(df['paid'])
df.loc[:,'hold'] = le.fit_transform(df['hold'])
df.loc[:,'draft'] = le.fit_transform(df['draft'])

In [17]:
df

,customer_id,date_paid,items,delivered,cancelled,paid,hold,draft,note
0,5d60cea34e77525ec5ca1413,None,None,0,0,0,0,1,
1,5d60ceba4e77525ec5ca1419,None,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,0,0,환완료
2,5d60d0a94e77525ec5ca1446,2019-09-06T09:13:26.749Z,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,0,1,0,0,
3,5d60e2694e77525ec5ca14b5,2019-10-21T02:18:23.528Z,[{'product_image': [{'id': {'$oid': '5da6ab300...,0,0,1,0,0,
4,5d60f3a74e77525ec5ca14da,None,None,0,0,0,0,1,
...,...,...,...,...,...,...,...,...,...
206684,5d70ccd419efa30eb2914271,2022-09-13T08:46:18.342Z,[{'product_image': [{'id': {'$oid': '6258da64c...,0,0,1,0,0,
206685,5d959ad20dabe405b156d820,None,[{'product_image': [{'id': {'$oid': '62b12b400...,0,0,0,0,1,
206686,5d918f180dabe405b156d5e5,2022-09-13T08:53:42.268Z,[{'product_image': [{'id': {'$oid': '620c7765d...,0,0,1,0,0,
206687,628ecb419d9388002406f007,2022-09-13T08:59:21.151Z,[{'product_image': [{'id': {'$oid': '630dc6166...,1,0,1,0,0,


In [ ]:
# status_id 끼리 묶어놓음
df.sort_values('status_id')

# status_id == '5d1d485b4b25b80d9fb1f372'는 버려도 될듯!
* boolean형 데이터 조합의 경우에 따라 status_id가 부여되는 것으로 추정 ex.배송중, 주문완료, 주문취소, 환불 등
* 그중 위의 해당되는 status_id의 경우만 보면, items는 all none, draft는 all 1 -> 따라서 그냥 drop해도 됨.
* 이러한 데이터가 총 13804개임. 얘는 확실히 drop해도 되겠음

In [ ]:
print('items 컬럼 none 개수(a): ', df['items'].isna().sum())
print('위의 status_id의 len(b): ',len(df[df['status_id'] == '5d1d485b4b25b80d9fb1f372']))
print("a - b = ", df['items'].isna().sum() - len(df[df['status_id'] == '5d1d485b4b25b80d9fb1f372']))
# df['items'].isna().sum() - len(df[df['status_id'] == '5d1d485b4b25b80d9fb1f372'])

In [ ]:
df[df['status_id'] == '5d1d485b4b25b80d9fb1f372']#.isna()  # 전부 none인뎁..?

In [ ]:
# status_id가 '5d1d485b4b25b80d9fb1f372' 것은 items가 모두 none이고, draft가 모두 1이다.
a = df['items'][df['status_id'] == '5d1d485b4b25b80d9fb1f372'].index  # items -> 전부 none
b = df['draft'][df['status_id'] == '5d1d485b4b25b80d9fb1f372'].index # draft -> 전부 1

# 인덱스 비교
s =  a == b

# False가 하나도 없음. all True
False in s

# paid와 draft 비교 -> paid와 draft는 반대 관계다!!!

## 1. 임시주문: paid ==0, draft == 1 -> 관심을 표한 것으로 봐도 될지..?

In [14]:
credit = df[['customer_id', 'items','cancelled', 'paid', 'hold', 'draft','note']][(df['cancelled']==0)& (df['draft']==1)]


In [15]:
# 임시주문 한 것 중에, 취소 X, items가 notna인 cuntomer_id들은 36356 -> 얘네는 관심을 표한 정도로 보면 될까?
credit[credit['customer_id'].notna() & credit['items'].notna()]

,customer_id,items,cancelled,paid,hold,draft,note
14,5d672e344e77525ec5ca17e3,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,0,0,1,
28,5d60cb524e77525ec5ca13d7,[{'product_image': [{'id': {'$oid': '62c2c0d71...,0,0,0,1,
38,5d6379ed4e77525ec5ca1634,[{'product_image': [{'id': {'$oid': '5e4a3e3c4...,0,0,0,1,
99,5d60d6ef4e77525ec5ca147b,[{'product_image': [{'id': {'$oid': '5dcb99472...,0,0,0,1,
103,5d61e52b4e77525ec5ca157e,[{'product_image': [{'id': {'$oid': '5d78577f1...,0,0,0,1,
...,...,...,...,...,...,...,...
206674,5e0ddf404267e105dfff04b7,[{'product_image': [{'id': {'$oid': '630dc6166...,0,0,0,1,
206675,5f8d6965512031633431ec7e,[{'product_image': [{'id': {'$oid': '623168e1f...,0,0,0,1,
206683,5e31033c4267e105dfff0e14,[{'product_image': [{'id': {'$oid': '630dc6166...,0,0,0,1,
206685,5d959ad20dabe405b156d820,[{'product_image': [{'id': {'$oid': '62b12b400...,0,0,0,1,


In [16]:
credit['items'][credit['customer_id'].notna() & credit['items'].notna()].value_counts

<bound method IndexOpsMixin.value_counts of 14        [{'product_image': [{'id': {'$oid': '5d60b74e4...
28        [{'product_image': [{'id': {'$oid': '62c2c0d71...
38        [{'product_image': [{'id': {'$oid': '5e4a3e3c4...
99        [{'product_image': [{'id': {'$oid': '5dcb99472...
103       [{'product_image': [{'id': {'$oid': '5d78577f1...
                                ...                        
206674    [{'product_image': [{'id': {'$oid': '630dc6166...
206675    [{'product_image': [{'id': {'$oid': '623168e1f...
206683    [{'product_image': [{'id': {'$oid': '630dc6166...
206685    [{'product_image': [{'id': {'$oid': '62b12b400...
206688    [{'product_image': [{'id': {'$oid': '61654cb14...
Name: items, Length: 36356, dtype: object>

## 2. 일반 거래: paid == 1, draft == 0

In [18]:
normal = df[['customer_id','cancelled', 'paid', 'hold', 'draft','note']][(df['paid']==1)& (df['draft']==0)]
normal

,customer_id,cancelled,paid,hold,draft,note
2,5d60d0a94e77525ec5ca1446,0,1,0,0,
3,5d60e2694e77525ec5ca14b5,0,1,0,0,
7,5d61ca9e4e77525ec5ca1573,0,1,0,0,
8,5d6348344e77525ec5ca15f5,0,1,0,0,
15,5d60cefc4e77525ec5ca1426,0,1,0,0,
...,...,...,...,...,...,...
206681,5e7c92adbfe4260944784430,0,1,0,0,
206682,5d9ef3790dabe405b156e346,0,1,0,0,
206684,5d70ccd419efa30eb2914271,0,1,0,0,
206686,5d918f180dabe405b156d5e5,0,1,0,0,


## 3. 취소 거래??: paid == 0, draft == 0

In [20]:
cancel= df[['customer_id','cancelled', 'paid', 'hold', 'draft','note']][(df['paid']==0)& (df['draft']==0)]
cancel

,customer_id,cancelled,paid,hold,draft,note
1,5d60ceba4e77525ec5ca1419,1,0,0,0,환완료
20,5d6c7c9519efa30eb2913e57,1,0,0,0,접수취소
50,5d67e3394e77525ec5ca182a,1,0,0,0,환완료
53,5d66390d4e77525ec5ca1732,1,0,0,0,환완료
60,5d6c60e519efa30eb2913e42,1,0,0,0,이체 내역 없음. 계속 연락 안 됨.
...,...,...,...,...,...,...
206583,61836e13bc63410023473e74,0,0,0,0,
206613,5f33869ca0b91001bf48c428,0,0,0,0,
206619,5d6677794e77525ec5ca17aa,0,0,0,0,
206665,5f6f7fcb48060c52caf15891,0,0,0,0,


In [22]:
a = cancel[(cancel['note'] != '') & (cancel['cancelled'] == 0)]
a

,customer_id,cancelled,paid,hold,draft,note
159,5d60cddd4e77525ec5ca140c,0,0,0,0,2019-11-19 11:53 운영자가 알림톡 재발송 함.
367,5d60d8cf4e77525ec5ca148c,0,0,0,0,2019-10-25 14:35 운영자가 알림톡 재발송 함.
1236,5d96ab5d0dabe405b156db10,0,0,0,0,2019-10-17 10:32 운영자가 알림톡 재발송 함.
1266,5d8eb0ab0dabe405b156d592,0,0,0,0,2019-10-17 10:32 운영자가 알림톡 재발송 함.
1345,5d931eea0dabe405b156d68a,0,0,0,0,2019-10-17 10:32 운영자가 알림톡 재발송 함.
...,...,...,...,...,...,...
138149,619b0525bc63410023478c22,0,0,0,0,2022-05-10 23:37:12 주문서 보관처리 (고객ID: 5f69c6ab48...
138269,619b0525bc63410023478c22,0,0,0,0,2022-05-11 11:52:23 주문서 보관처리 (고객ID: 5dbff37c2b...
170045,619b0525bc63410023478c22,0,0,0,0,2022-06-23 11:33:59 주문서 보관처리 (고객ID: 5f33869ca0...
192762,5e5628d4bfe4260944782942,0,0,0,0,"2022-08-10 정기결제 상품에 대한 주문서 수동 생성 (계윤재, 임영서, 이동섭)"


In [23]:
# 접수 취소 or 환불
cancel[(cancel['cancelled'] == 1)]

,customer_id,cancelled,paid,hold,draft,note
1,5d60ceba4e77525ec5ca1419,1,0,0,0,환완료
20,5d6c7c9519efa30eb2913e57,1,0,0,0,접수취소
50,5d67e3394e77525ec5ca182a,1,0,0,0,환완료
53,5d66390d4e77525ec5ca1732,1,0,0,0,환완료
60,5d6c60e519efa30eb2913e42,1,0,0,0,이체 내역 없음. 계속 연락 안 됨.
...,...,...,...,...,...,...
205642,5ebb5aa409982e0735b2d76b,1,0,0,0,고객이 입금 전 직접 취소함.
205644,6079a543c1908b001a63e77d,1,0,0,0,2022-09-10 00:00:00 입금기한 만료로 자동취소 처리
205652,5d6f7b5419efa30eb2914111,1,0,0,0,2022-09-10 00:00:00 입금기한 만료로 자동취소 처리
205656,5d6679b94e77525ec5ca17ab,1,0,0,0,2022-09-10 00:00:01 입금기한 만료로 자동취소 처리


In [ ]:
cancel_maybe['status_id'][(cancel_maybe['cancelled'] == 1)].value_counts()

In [ ]:
# 임시거래 + 일반거래 + 취소거래 = 206689 로 정확히 일치
63013 + 140093 + 3583

# case

In [ ]:
# paid, hold 둘 다 1인데 closed만 다름. 무슨 차이지?
groups = df_origin[['status_id','note']][(df_origin['hold'] == 1) & (df_origin['cancelled'] == 0)]

# paid == 1인 것만 사용하라고..?? ㅎㅅㅎ (paid==1이면 draft == 0)

In [22]:
# 돈 낸 경우 : 1개 값만 빼고 date_paid가 모두 있음
# df[df['paid'] == 1]

# date_paid가 none이고 paid == 1인 값은 단 1개 -> 결국 paid된 값임. 아래 시간으로 채워넣을 수 있을 듯!
# df[(df['date_paid'].isna() == True) & (df['paid'] == 1)].note.values

# 취소 또는 환불한 경우
# df[df['cancelled'] == 1]

# 돈내고 취소한 경우
# df[(df['paid'] == 1) & (df['cancelled'] == 1)]

# 돈 내고 취고하지 않은 완전한 paid
df[(df['paid'] == 1) & (df['cancelled'] == 0)]

# 돈 내고 배달까지 됐는데 취소한 경우
# df[(df['cancelled'] == 1) & (df['paid'] == 1) & (df['delivered'] == 1)]

# 돈 내고 배달 완료한 경우 -> 거래 완료 : 35792
# df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 1)]

# 돈 내고 배달 안한 경우: 101750개. 상품군으로 살펴보자
# df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 0)]

# 돈 내고 배달 안했는데, hold == 1인 경우
# print('돈 내고 배달 안했는데, hold == 1인 경우')
# print(len(df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 0) & (df['hold'] == 1)]),'개')
# df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 0) & (df['hold'] == 1)]

# 돈 내고 배달 했는데, hold == 1인 경우
# print('돈 내고 배달 했는데, hold == 1인 경우')
# print(len(df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 1) & (df['hold'] == 1)]),'개')
# df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 1) & (df['hold'] == 1)]

,customer_id,date_paid,items,delivered,cancelled,paid,hold,draft,note
2,5d60d0a94e77525ec5ca1446,2019-09-06T09:13:26.749Z,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,0,1,0,0,
3,5d60e2694e77525ec5ca14b5,2019-10-21T02:18:23.528Z,[{'product_image': [{'id': {'$oid': '5da6ab300...,0,0,1,0,0,
7,5d61ca9e4e77525ec5ca1573,2019-10-07T00:15:36.289Z,[{'product_image': [{'id': {'$oid': '5d96c42e0...,0,0,1,0,0,
8,5d6348344e77525ec5ca15f5,2019-08-28T08:47:23.961Z,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,0,1,0,0,
15,5d60cefc4e77525ec5ca1426,2019-09-03T05:02:22.622Z,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,0,1,0,0,
...,...,...,...,...,...,...,...,...,...
206681,5e7c92adbfe4260944784430,2022-09-13T08:35:46.208Z,[{'product_image': [{'id': {'$oid': '62c6a46fb...,1,0,1,0,0,
206682,5d9ef3790dabe405b156e346,2022-09-13T08:39:57.460Z,[{'product_image': [{'id': {'$oid': '5f3e3111a...,0,0,1,0,0,
206684,5d70ccd419efa30eb2914271,2022-09-13T08:46:18.342Z,[{'product_image': [{'id': {'$oid': '6258da64c...,0,0,1,0,0,
206686,5d918f180dabe405b156d5e5,2022-09-13T08:53:42.268Z,[{'product_image': [{'id': {'$oid': '620c7765d...,0,0,1,0,0,


In [ ]:
# 전체 돈 낸 것 - ㄷ
len(df[df['paid'] == 1]) - 

# 돈 내고 배달 안한 경우: 101750개. 상품군으로 살펴보자
delivered == 0이라고 해서 배송이 필요하지 않은 상품(ex.강의 등)은 아님.  
paid가 됐다면 delivered의 여부는 무시해도 될 것 같음..?

In [ ]:
p1d0 = df[(df['cancelled'] == 0) & (df['paid'] == 1) & (df['delivered'] == 0)]
p1d0 = p1d0['items'].to_frame()
p1d0[:5].values

In [ ]:
# 김다솔 이름 들어간 인덱스 찾기 -> 같은 고객 아님. 기업 내부 인물로 추정
a = []
for i in df['note'][df['hold'] == 1].index:
    if df['note'].iloc[i].find('김다솔'):
        a.append(i)
# print(len(a),'개', a)

# paid == 1인것만 분석 해보자

In [65]:
complete_paid = df[df['paid'] == 1]

In [33]:
complete_paid.columns

Index(['customer_id', 'date_paid', 'items', 'cancelled', 'paid', 'hold',
       'draft', 'note'],
      dtype='object')

In [66]:
complete_paid = complete_paid[['customer_id', 'date_paid', 'items', 'cancelled', 'paid', 'draft', 'note']]

In [67]:
# date 조회하기 위해 전처리
complete_paid['date_paid'] = complete_paid['date_paid'].apply(lambda x: x.replace('T', ' ')[:-5] if type(x)==str else None)

In [68]:
complete_paid

,customer_id,date_paid,items,cancelled,paid,draft,note
2,5d60d0a94e77525ec5ca1446,2019-09-06 09:13:26,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,
3,5d60e2694e77525ec5ca14b5,2019-10-21 02:18:23,[{'product_image': [{'id': {'$oid': '5da6ab300...,0,1,0,
7,5d61ca9e4e77525ec5ca1573,2019-10-07 00:15:36,[{'product_image': [{'id': {'$oid': '5d96c42e0...,0,1,0,
8,5d6348344e77525ec5ca15f5,2019-08-28 08:47:23,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,
15,5d60cefc4e77525ec5ca1426,2019-09-03 05:02:22,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,
...,...,...,...,...,...,...,...
206681,5e7c92adbfe4260944784430,2022-09-13 08:35:46,[{'product_image': [{'id': {'$oid': '62c6a46fb...,0,1,0,
206682,5d9ef3790dabe405b156e346,2022-09-13 08:39:57,[{'product_image': [{'id': {'$oid': '5f3e3111a...,0,1,0,
206684,5d70ccd419efa30eb2914271,2022-09-13 08:46:18,[{'product_image': [{'id': {'$oid': '6258da64c...,0,1,0,
206686,5d918f180dabe405b156d5e5,2022-09-13 08:53:42,[{'product_image': [{'id': {'$oid': '620c7765d...,0,1,0,


In [75]:
complete_paid['date_paid'] = pd.to_datetime(complete_paid['date_paid'], format='%Y-%m-%d %H:%M:%S', errors='ignore')

In [77]:
# 날짜 추출하기
complete_paid['order_date']=complete_paid['date_paid'].dt.date
 
# 시간 추출하기
complete_paid['order_time']=complete_paid['date_paid'].dt.time
 
# 시 추출하기
# df['log_hour']=df['dt2'].dt.hour
 
# 분 추출하기
# df['log_min']=df['dt2'].dt.minute

In [78]:
complete_paid

,customer_id,date_paid,items,cancelled,paid,draft,note,order_date,order_time
2,5d60d0a94e77525ec5ca1446,2019-09-06 09:13:26,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,,2019-09-06,09:13:26
3,5d60e2694e77525ec5ca14b5,2019-10-21 02:18:23,[{'product_image': [{'id': {'$oid': '5da6ab300...,0,1,0,,2019-10-21,02:18:23
7,5d61ca9e4e77525ec5ca1573,2019-10-07 00:15:36,[{'product_image': [{'id': {'$oid': '5d96c42e0...,0,1,0,,2019-10-07,00:15:36
8,5d6348344e77525ec5ca15f5,2019-08-28 08:47:23,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,,2019-08-28,08:47:23
15,5d60cefc4e77525ec5ca1426,2019-09-03 05:02:22,[{'product_image': [{'id': {'$oid': '5d60b74e4...,0,1,0,,2019-09-03,05:02:22
...,...,...,...,...,...,...,...,...,...
206681,5e7c92adbfe4260944784430,2022-09-13 08:35:46,[{'product_image': [{'id': {'$oid': '62c6a46fb...,0,1,0,,2022-09-13,08:35:46
206682,5d9ef3790dabe405b156e346,2022-09-13 08:39:57,[{'product_image': [{'id': {'$oid': '5f3e3111a...,0,1,0,,2022-09-13,08:39:57
206684,5d70ccd419efa30eb2914271,2022-09-13 08:46:18,[{'product_image': [{'id': {'$oid': '6258da64c...,0,1,0,,2022-09-13,08:46:18
206686,5d918f180dabe405b156d5e5,2022-09-13 08:53:42,[{'product_image': [{'id': {'$oid': '620c7765d...,0,1,0,,2022-09-13,08:53:42


In [81]:
complete_paid['items'][:2].values

array([list([{'product_image': [{'id': {'$oid': '5d60b74e4e77525ec5ca137c'}, 'alt': '', 'position': 99, 'filename': '제일경옥고3.png', 'url': 'https://warehouse.medistream.co.kr/images/products/5d5cd0f74e77525ec5ca12ab/제일경옥고3.png'}], 'id': {'$oid': '5d60d2d04e77525ec5ca145c'}, 'product_id': {'$oid': '5d5cd0f74e77525ec5ca12ab'}, 'variant_id': None, 'quantity': 1, 'discount_total': 0, 'name': '제일 경옥고', 'price': 59400, 'price_total': 59400, 'sku': '', 'tax_class': '과세', 'tax_total': 0, 'variant_name': '', 'weight': 0}]),
       list([{'product_image': [{'id': {'$oid': '5da6ab300dabe405b156eb62'}, 'alt': '', 'position': 99, 'filename': '기획자.png', 'url': 'https://warehouse.medistream.co.kr/images/products/5da696740dabe405b156eb5c/기획자.png'}, {'id': {'$oid': '5da6ab360dabe405b156eb63'}, 'alt': '', 'position': 99, 'filename': '기획자.png', 'url': 'https://warehouse.medistream.co.kr/images/products/5da696740dabe405b156eb5c/기획자.png'}], 'id': {'$oid': '5dad13270dabe405b156eee5'}, 'product_id': {'$oid': '

In [1]:
df1

NameError: name 'df1' is not defined

In [6]:
df = df.head(200)
df.to_csv()

,_id,date_created,date_placed,date_updated,date_closed,date_paid,date_cancelled,number,shipping_status,items,...,channel,note,comments,coupon,tracking_number,customer_id,status_id,payment_method_id,shipping_method_id,tags
0,{'$oid': '5d60cebb4e77525ec5ca141c'},{'$date': '2019-08-24T05:44:27.710Z'},None,None,None,None,None,1036,,[],...,,,,,,{'$oid': '5d60cea34e77525ec5ca1413'},{'$oid': '5d1d485b4b25b80d9fb1f372'},None,None,[]
1,{'$oid': '5d60daf14e77525ec5ca1497'},{'$date': '2019-08-24T06:36:33.098Z'},{'$date': '2019-08-24T06:38:22Z'},{'$date': '2019-09-22T12:14:33.718Z'},None,None,None,1072,,[{'product_image': [{'id': {'$oid': '5d60b74e4...,...,,환완료,,,,{'$oid': '5d60ceba4e77525ec5ca1419'},{'$oid': '5d53f7cb4e77525ec5ca116d'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},None,[]
2,{'$oid': '5d60d2d04e77525ec5ca145b'},{'$date': '2019-08-24T06:01:52.756Z'},{'$date': '2019-09-04T08:18:26Z'},{'$date': '2019-09-06T09:13:52.571Z'},None,{'$date': '2019-09-06T09:13:26.749Z'},None,1056,,[{'product_image': [{'id': {'$oid': '5d60b74e4...,...,,,,,,{'$oid': '5d60d0a94e77525ec5ca1446'},{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[]
3,{'$oid': '5d60e27c4e77525ec5ca14b6'},{'$date': '2019-08-24T07:08:44.815Z'},{'$date': '2019-10-21T02:09:35Z'},{'$date': '2019-10-21T05:14:24.764Z'},None,{'$date': '2019-10-21T02:18:23.528Z'},None,1081,,[{'product_image': [{'id': {'$oid': '5da6ab300...,...,,,,,,{'$oid': '5d60e2694e77525ec5ca14b5'},{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[]
4,{'$oid': '5d60f8764e77525ec5ca14e5'},{'$date': '2019-08-24T08:42:30.087Z'},None,None,None,None,None,1096,,[],...,,,,,,{'$oid': '5d60f3a74e77525ec5ca14da'},{'$oid': '5d1d485b4b25b80d9fb1f372'},None,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,{'$oid': '5d663a824e77525ec5ca173b'},{'$date': '2019-08-28T08:25:38.341Z'},{'$date': '2019-10-07T00:52:06Z'},{'$date': '2019-10-07T02:31:34.933Z'},None,{'$date': '2019-10-07T02:17:26.653Z'},None,1285,,[{'product_image': [{'id': {'$oid': '5d96c42e0...,...,,,,,,{'$oid': '5d65e2cb4e77525ec5ca1709'},{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[]
196,{'$oid': '5d686d0b19efa30eb2913d56'},{'$date': '2019-08-30T00:25:47.490Z'},{'$date': '2019-08-30T00:28:13Z'},{'$date': '2019-09-22T15:45:20.190Z'},None,None,None,1357,,[{'product_image': [{'id': {'$oid': '5d60b74e4...,...,,환완료,,,,{'$oid': '5d67d12d4e77525ec5ca1828'},{'$oid': '5d53f7cb4e77525ec5ca116d'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},None,[]
197,{'$oid': '5d689cc419efa30eb2913d85'},{'$date': '2019-08-30T03:49:24.211Z'},None,None,None,None,None,1374,,[],...,,,,,,{'$oid': '5d66402c4e77525ec5ca1750'},{'$oid': '5d1d485b4b25b80d9fb1f372'},None,None,[]
198,{'$oid': '5d68bdf019efa30eb2913da9'},{'$date': '2019-08-30T06:10:56.103Z'},{'$date': '2019-08-30T06:15:22Z'},{'$date': '2019-09-22T11:31:42.300Z'},None,{'$date': '2019-09-01T22:21:00.314Z'},None,1385,,[{'product_image': [{'id': {'$oid': '5d60b74e4...,...,,,,,,{'$oid': '5d68bde919efa30eb2913da8'},{'$oid': '5d48eb4a4e77525ec5ca107a'},{'$oid': '5d23e5dc4b25b80d9fb1f3e4'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[]


# unwrap 이상한 점

_id(5d87c6de0dabe405b156d441) -> items의 value가 1개인데

In [57]:
df1[df1['_id']=={'$oid': '630734ce68554000236d8559'}]['items']

199250    [{'product_image': [{'id': {'$oid': '5f1128e02...
Name: items, dtype: object

paid == 1이지만 date_paid가 none이라서,, note보고 날짜 확인 가능

In [58]:
# bb = df1[df1['_id'] == {'$oid': '630734ce68554000236d8559'}]
df1['_id'] = df1['_id'].apply(lambda x: None if x == None else x.get('$oid',None))

In [80]:
len(df1[df1['_id']=='5f586700a0b91001bf48f95c']['items'].iloc[0])

15

In [76]:
for i in df1[df1['_id']=='5fcdbcf31c6ef3001a114aa0']['items']:
    display(len(i), i)

13

[{'product_image': [{'id': {'$oid': '5fb769a85120316334324ac9'},
    'alt': '',
    'position': 99,
    'filename': '2020-11-20T16-00-54.557Z.jpg',
    'url': 'https://warehouse.medistream.co.kr/images/products/5fb769245120316334324a0a/2020-11-20T16-00-54.557Z.jpg'}],
  'id': {'$oid': '5fcdbd2a0204ef001a67f0e5'},
  'product_id': {'$oid': '5fb769245120316334324a0a'},
  'variant_id': None,
  'quantity': 2,
  'custom_price': {},
  'custom_note': {},
  'discount_total': 0,
  'name': '[씨케이] 길경[한국] 600g, 한국',
  'price': 15750,
  'price_total': 31500,
  'sku': 'MD-HM-A025-CK087-OTH',
  'tax_class': '면세',
  'tax_total': 0,
  'variant_name': '',
  'weight': 0.6000000000000001},
 {'product_image': [{'id': {'$oid': '5fb769dc5120316334324ade'},
    'alt': '',
    'position': 99,
    'filename': '2020-11-20T16-01-46.102Z.jpg',
    'url': 'https://warehouse.medistream.co.kr/images/products/5fb7692e5120316334324a1c/2020-11-20T16-01-46.102Z.jpg'}],
  'id': {'$oid': '5fcdbd3d1c6ef3001a114aa7'},
  'prod

In [66]:
len(df1[df1['_id']=='5d87c6de0dabe405b156d441']['items'])

1

In [38]:
bb['note'].values

array(['정현섭님 [22년8월26일 14시41분] 5사단에서 무통장입금으로 주문완료하였습니다.\n제품 검수후 입금되는 절차기 때문에 결제완료로 변경후 제품 발주 부탁드리겠습니다.\n\n제품 검수후 입금되는 절차기 때문에 결제완료로 변경후 제품 발주 요청이 들어와서 주문상태 입금대기 > 결제완료로 변경하겠습니다.<br/>정현섭님 [22년8월29일 11시35분] (발주됨) => (입금대기) 직접 상태변경'],
      dtype=object)

In [39]:
bb

,_id,date_created,date_placed,date_updated,date_closed,date_paid,date_cancelled,number,shipping_status,items,...,channel,note,comments,coupon,tracking_number,customer_id,status_id,payment_method_id,shipping_method_id,tags
199250,{'$oid': '630734ce68554000236d8559'},{'$date': '2022-08-25T08:37:34.085Z'},{'$date': '2022-08-26T02:24:35Z'},{'$date': '2022-09-02T00:09:37.606Z'},None,None,None,269617,,[{'product_image': [{'id': {'$oid': '5f1128e02...,...,주봉현,정현섭님 [22년8월26일 14시41분] 5사단에서 무통장입금으로 주문완료하였습니다...,,,,{'$oid': '5e7c41ffbfe426094478435c'},{'$oid': '5d1d61954b25b80d9fb1f379'},{'$oid': '5d48ead24e77525ec5ca1079'},{'$oid': '5cf8bbba0098b2225c5dfab1'},[]


In [46]:
print(len(bb['items'].values[0])) # 35개 맞음.
print(bb['items'].values[0])
# len(bb['items'].values[0])

35
[{'product_image': [{'id': {'$oid': '5f1128e0221c8b5f77c8ee90'}, 'alt': '', 'position': 1, 'filename': '동방-스프링침-1,000쌈.jpg', 'url': 'https://warehouse.medistream.co.kr/images/products/5f10f2a7221c8b5f77c8e7c2/동방-스프링침-1,000쌈.jpg'}], 'id': {'$oid': '6307350b68554000236d855e'}, 'product_id': {'$oid': '5f10f2a7221c8b5f77c8e7c2'}, 'variant_id': {'$oid': '5f10f374221c8b5f77c8e9d6'}, 'quantity': 4, 'custom_price': {}, 'custom_note': {}, 'origin_items': [{'id': 'MEAMDBDBDBC1H254G010', 'group_quantity': 1}], 'category_id': '5e91ce6fbfe42609447852d1', 'category_ids': [{'$oid': '5fa3d1885120316334321da0'}, {'$oid': '5fa3d1c95120316334321da1'}, {'$oid': '5ee19600e510ee503167cb3d'}, {'$oid': '60629b7fc1908b001a63ca31'}], 'discount_total': 0, 'name': '동방 스프링침 한쌈용 1,000쌈 (10,000pcs)', 'price': 156000, 'price_total': 624000, 'sku': 'ME-AM-DBDB-DBC1H-254', 'tax_class': '과세', 'tax_total': 0, 'variant_name': '옵션: 0.25x40mm', 'weight': 0, 'price_detail': {'couponDiscountAmount': 62400, 'cardDiscountAmo

# log와 연결

In [14]:
df[df['customer_id'] == {'$oid': '6225cd53f967570023fa9c23'}]['items']

115265                                                   []
115266    [{'product_image': [{'id': {'$oid': '5e4ccb5a4...
115267                                                   []
119579    [{'product_image': [{'id': {'$oid': '60334f285...
123932                                                   []
127246    [{'product_image': [{'id': {'$oid': '5d9edeef0...
185930    [{'product_image': [{'id': {'$oid': '62c2a65a8...
185931    [{'product_image': [{'id': {'$oid': '61dfd8037...
185932    [{'product_image': [{'id': {'$oid': '61dfd8037...
185933    [{'product_image': [{'id': {'$oid': '61dfd8037...
186097    [{'product_image': [{'id': {'$oid': '61dfd8037...
186098    [{'product_image': [{'id': {'$oid': '61dfd8037...
187502    [{'product_image': [{'id': {'$oid': '61dfd8037...
187503    [{'product_image': [{'id': {'$oid': '61dfd8037...
187504    [{'product_image': [{'id': {'$oid': '61dfd8037...
187761    [{'product_image': [{'id': {'$oid': '62d8dbc2c...
187973    [{'product_image': [{'id': {'$